<a href="https://colab.research.google.com/github/ProgramasParaFisicaDeParticulas/lecturadedatosabiertoscmszmumu-MPM-cvr/blob/main/CMSOpenDataZtoMuMu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
#panda nos permite manipular el archivo csv https://pandas.pydata.org
import pandas as pd
#numpy nos permite hacer operaciones usando vectores https://numpy.org
import numpy as np
#plotly nos permite crear graficas interactivas https://plotly.com/python/
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from scipy.optimize import curve_fit
from scipy.signal import fftconvolve

%matplotlib inline
#usamos la liga que está directamente en la página.
particles = pd.read_csv('https://opendata.cern.ch/record/5208/files/Zmumu.csv',delimiter=',')
particles



,Run,Event,pt1,eta1,phi1,Q1,dxy1,iso1,pt2,eta2,phi2,Q2,dxy2,iso2
0,165617,74969122,54.7055,-0.4324,2.5742,1,-0.0745,0.4999,34.2464,-0.9885,-0.4987,-1,0.0712,3.4221
1,165617,75138253,24.5872,-2.0522,2.8666,-1,-0.0554,0.0000,28.5389,0.3852,-1.9912,1,0.0515,0.0000
2,165617,75887636,31.7386,-2.2595,-1.3323,-1,0.0879,0.0000,30.2344,-0.4684,1.8833,1,-0.0876,0.0000
3,165617,75779415,39.7394,-0.7123,-0.3123,1,0.0585,0.0000,48.2790,-0.1956,2.9703,-1,-0.0492,0.0000
4,165617,75098104,41.2998,-0.1571,-3.0408,1,-0.0305,1.2280,43.4508,0.5910,-0.0428,-1,0.0442,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,173692,1245050481,8.9721,-1.6158,-2.7176,1,-0.0294,8.7183,13.5266,1.8709,2.7911,1,-0.0689,0.4202
9996,173692,1244979327,39.4543,-1.1310,-2.0568,1,0.0417,0.0000,47.7209,-0.1834,1.7965,-1,-0.0794,0.0000
9997,173692,1245305163,40.6034,-1.5051,-1.6758,-1,0.0635,0.2012,44.4382,-1.0010,1.5380,1,-0.0671,1.1499
9998,173692,1245041468,37.2853,1.0024,-2.1486,1,0.0361,0.0000,41.0945,-0.1200,0.8850,-1,-0.0291,0.0000


In [94]:
df=particles

fig_5 = px.histogram(
    particles,
    x='pt2',
    nbins=10000,
    title='Histograma momento transversal 2',
    labels={'pt2': 'Momento transversal pt2 [GeV/c]', 'count': 'Frecuencia'},
    opacity=0.75,
    color_discrete_sequence=['#636EFA']
)

fig_5.update_layout(
    xaxis_title='pt2 [GeV/c]',
    yaxis_title='Frecuencia',
    bargap=0.05,
    font=dict(family='Arial', size=14),
    title_font=dict(size=18, family='Arial', color='darkblue')
)

fig_5.show()

In [95]:
fig_5 = px.histogram(
    particles,
    x='pt1',
    nbins=10000,
    title='Histograma momento transversal 1',
    labels={'pt1': 'Momento transversal pt1 [GeV/c]', 'count': 'Frecuencia'},
    opacity=0.75,
    color_discrete_sequence=['#636EFA']
)

fig_5.update_layout(
    xaxis_title='pt1 [GeV/c]',
    yaxis_title='Frecuencia',
    bargap=0.05,
    font=dict(family='Arial', size=14),
    title_font=dict(size=18, family='Arial', color='darkblue')
)

fig_5.show()

In [96]:
# Cuenta cada valor de Q1 y Q2 por separado
q1 = particles["Q1"].value_counts().sort_index() # Series: índice = carga, valor = frecuencia
q2 = particles["Q2"].value_counts().sort_index()

#  Organiza en una tabla ordenada (-1 primero, luego +1)
charges = [-1, 1] # orden explícito
table = pd.DataFrame({
    "Charge": charges,
    "Q1_count": [q1.get(c, 0) for c in charges],
    "Q2_count": [q2.get(c, 0) for c in charges],
})

# Muestra la tabla
print(table.to_string(index=False))

 Charge  Q1_count  Q2_count
     -1      5156      4808
      1      4844      5192


In [97]:
particles["Q1"] = particles["Q1"].astype(int)
particles["Q2"] = particles["Q2"].astype(int)

# Cargar solo los valores de carga esperados (-1 y 1)
charges = [-1, 1]
q1_df = particles[particles["Q1"].isin(charges)]
q2_df = particles[particles["Q2"].isin(charges)]

# Histograma para Q1
fig_q1 = px.histogram(
    q1_df,
    x="Q1",
    category_orders={"Q1": charges},
    labels={"Q1": "Carga", "count": "Frecuencia"},
    title="Histograma de Q1",
    color_discrete_sequence=["#636EFA"],
    opacity=0.8
)
fig_q1.update_layout(xaxis_title="Carga", yaxis_title="Frecuencia", width=500, height=500,
    bargap=0.2)
fig_q1.show()

# Histograma para Q2
fig_q2 = px.histogram(
    q2_df,
    x="Q2",
    category_orders={"Q2": charges},
    labels={"Q2": "Carga", "count": "Frecuencia"},
    title="Histograma de Q2",
    color_discrete_sequence=["#636EFA"],
    opacity=0.8
)
fig_q2.update_layout(xaxis_title="Carga", yaxis_title="Frecuencia", width=500, height=500,
    bargap=0.2)
fig_q2.show()


In [98]:

df['QT'] = particles['Q1'] + particles['Q2']

df[["Event","Run","QT"]]

,Event,Run,QT
0,74969122,165617,0
1,75138253,165617,0
2,75887636,165617,0
3,75779415,165617,0
4,75098104,165617,0
...,...,...,...
9995,1245050481,173692,2
9996,1244979327,173692,0
9997,1245305163,173692,0
9998,1245041468,173692,0


In [99]:
# Cuenta cada valor de Q1 y Q2 por separado
QT = df["QT"].value_counts().sort_index() # Series: índice = carga, valor = frecuencia

 #  Organiza en una tabla ordenada
charges2 = [0, 2, -2] # orden explícito
table3 = pd.DataFrame({
    "Value": charges2,
    "QT_count": [QT.get(c, 0) for c in charges2],

})

# Muestra la tabla
print(table3.to_string(index=False))

 Value  QT_count
     0      9664
     2       186
    -2       150


In [100]:
#Calculamos masa invariante por partes (para entenderle)

df["pt1*pt2"]=particles['pt1'] * particles['pt2']
df["eta1-eta2"]=particles['eta1']-particles['eta2']
df["phi1-phi2"]=particles['phi1']-particles['phi2']

df["cosh"]=np.cosh(df['eta1-eta2'])
df["cos"]=np.cos(df['phi1-phi2'])

df["M^2"]=2*df["pt1*pt2"]*(df["cosh"]-df["cos"])
df["M"]=np.sqrt(df["M^2"])
df["M_r"]=np.round(df["M"])

df=df.drop(columns=["pt1","pt2","eta1","eta2","Q1","Q2","dxy1","dxy2","phi1","phi2","iso1","iso2","QT"])
df


,Run,Event,pt1*pt2,eta1-eta2,phi1-phi2,cosh,cos,M^2,M,M_r
0,165617,74969122,1873.466435,0.5561,3.0729,1.158650,-0.997642,8079.478521,89.885919,90.0
1,165617,75138253,701.691642,-2.4374,4.8578,5.765319,0.144899,7887.602846,88.812177,89.0
2,165617,75887636,959.597528,-1.7911,-3.2156,3.081411,-0.997263,7827.769550,88.474683,88.0
3,165617,75779415,1918.578493,-0.5167,-3.2826,1.136486,-0.990075,8159.947752,90.332429,90.0
4,165617,75098104,1794.509350,-0.7481,-2.9980,1.293123,-0.989708,8193.124926,90.515882,91.0
...,...,...,...,...,...,...,...,...,...,...
9995,173692,1245050481,121.362008,-3.4867,-5.5087,16.354267,0.714781,3796.078838,61.612327,62.0
9996,173692,1244979327,1882.794705,-0.9476,-3.8533,1.483591,-0.757248,8438.078560,91.859015,92.0
9997,173692,1245305163,1804.342010,-0.5041,-3.2138,1.129772,-0.997394,7676.270391,87.614328,88.0
9998,173692,1245041468,1532.220761,1.1224,-3.0336,1.698858,-0.994174,8252.641020,90.844048,91.0


In [101]:
df["M_r"].describe()

,M_r
count,10000.000000
mean,87.973100
std,8.843427
min,60.000000
25%,87.000000
50%,90.000000
75%,92.000000
max,120.000000


In [102]:
fig = px.histogram(
    df,
    x="M_r",
    nbins=len(df["M_r"].unique()),
    title="Histograma de Masa Invariante (M_r)",
    labels={"M_r": "M_r [GeV/c^2]", "count": "Frecuencia"},
    opacity=0.75,
    color_discrete_sequence=["#636EFA"]
)

fig.update_layout(
    bargap=0.05,
    xaxis_title="M_r [GeV/c^2]",
    yaxis_title="Frecuencia"
)

fig.show()

In [132]:
df["p_x1"]=particles['pt1']*np.cos(particles['phi1'])
df["p_x2"]=particles['pt2']*np.cos(particles['phi2'])


df["p_y1"]=particles['pt1']*np.sin(particles['phi1'])
df["p_y2"]=particles['pt2']*np.sin(particles['phi2'])


df["p_z1"]=particles['pt1']*np.sinh(particles['eta1'])
df["p_z2"]=particles['pt2']*np.sinh(particles['eta2'])

df["p_x"]=df['p_x1']+df['p_x2']
df["p_y"]=df['p_y1']+df['p_y2']
df["p_z"]=df['p_z1']+df['p_z2']

m_mu = 0.105658 # GeV
df["E_1"]=np.sqrt((particles['pt1']*np.cosh(particles['eta1']))**2+(m_mu)**2)
df["E_2"]=np.sqrt((particles['pt2']*np.cosh(particles['eta2']))**2+(m_mu)**2)
df["E"]=df["E_1"]+df["E_2"]

df["m^2"]=df["E"]**2-df["p_x"]**2-df["p_y"]**2-df["p_z"]**2
df["m"]=np.sqrt(df["m^2"])
df["m_r"]=np.round(df["m"])


df[['Event','Run','p_x1','p_x2','p_y1','p_y2','p_z1','p_z2','p_x','p_y','p_z','E_1','E_2','E','m^2','m','m_r']]

,Event,Run,p_x1,p_x2,p_y1,p_y2,p_z1,p_z2,p_x,p_y,p_z,E_1,E_2,E,m^2,m,m_r
0,74969122,165617,-46.133428,30.075362,29.400655,-16.379515,-24.398698,-39.641341,-16.058066,13.021141,-64.040039,59.899911,52.385714,112.285625,8079.523376,89.886169,90.0
1,75138253,165617,-23.663390,-11.647553,6.676405,-26.053854,-94.126653,11.267068,-35.310943,-19.377449,-82.859585,97.284985,30.682679,127.967664,7887.664090,88.812522,89.0
2,75887636,165617,7.497983,-9.295325,-30.840217,28.770052,-150.344100,-14.685350,-1.797341,-2.070165,-165.029450,153.657731,33.612343,187.270074,7827.845354,88.475111,88.0
3,75779415,165617,37.817182,-47.572449,-12.209859,8.229456,-30.761477,-9.503704,-9.755267,-3.980403,-40.265181,50.254348,49.205624,99.459972,8159.992412,90.332676,90.0
4,75098104,165617,-41.090192,43.411009,-4.155672,-1.859127,-6.514920,27.200637,2.320817,-6.014798,20.685717,41.810631,51.262636,93.073266,8193.170046,90.516131,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1245050481,173692,-8.177653,-12.704232,-3.691147,4.644501,-21.681888,42.880360,-20.881885,0.953353,21.198472,23.465166,44.963378,68.428544,3796.128383,61.612729,62.0
9996,1244979327,173692,-18.428948,-10.679567,-34.885752,46.510549,-54.763423,-8.801159,-29.108515,11.624797,-63.564582,67.495818,48.525827,116.021645,8438.124441,91.859264,92.0
9997,1245305163,173692,-4.255676,1.457148,-40.379764,44.414303,-86.944243,-52.292424,-2.798527,4.034540,-139.236666,95.958056,68.624065,164.582122,7676.316312,87.614590,88.0
9998,1245041468,173692,-20.364689,26.024715,-31.232564,31.803649,43.955938,-4.943184,5.660026,0.571086,39.012754,57.639650,41.390870,99.030520,8252.686910,90.844300,91.0


In [104]:
fig = px.histogram(
    df,
    x="m_r",
    nbins=len(df["m_r"].unique()),
    title="Histograma de Masa Invariante (m_r)",
    labels={"m_r": "m_r [GeV/c^2]", "count": "Frecuencia"},
    opacity=0.75,
    color_discrete_sequence=["#636EFA"]
)

fig.update_layout(
    bargap=0.05,
    xaxis_title="m_r [GeV/c^2]",
    yaxis_title="Frecuencia"
)

fig.show()

In [133]:
masas = df["m"].values

counts, bins = np.histogram(masas, bins=np.arange(60, 120.5, 0.5))
centers = 0.5 * (bins[:-1] + bins[1:])

def breit_wigner(m, M, Gamma):
    eps = 1e-9
    num = M**2*Gamma**2
    denom = (m**2-M**2)**2+M**2*Gamma**2+eps
    return num/denom

def model(m, M, Gamma, N, a, b):
    return N * breit_wigner(m, M, Gamma) + a + b * m

# Ajustar en la región 80–100 GeV
mask = (centers > 80) & (centers < 100)
x_fit = centers[mask]
y_fit = counts[mask]
sigma = np.sqrt(np.where(y_fit > 0, y_fit, 1))

p0 = [91.2, 2.5, y_fit.max()*5, y_fit.min(), 0.0]

popt, pcov = curve_fit(model, x_fit, y_fit, p0=p0, sigma=sigma, absolute_sigma=True, maxfev=10000)


M_Z, Gamma_Z, N_sig, a_bg, b_bg = popt
dM_Z, dGamma_Z = np.sqrt(np.diag(pcov))[:2]

print(f"M_Z = {M_Z:.3f} ± {dM_Z:.3f} GeV")
print(f"Gamma_Z = {Gamma_Z:.3f} ± {dGamma_Z:.3f} GeV")

# Creamos gráfica
m_plot = np.linspace(60, 120, 2000)

hist = go.Histogram(
    x=df["m_r"],
    xbins=dict(start=60, end=120, size=0.5),
    name="Datos",
    marker_color="lightgray",
    opacity=0.75,
)

# Señal + Fondo
modelo = go.Scatter(
    x=m_plot,
    y=model(m_plot, *popt),
    mode="lines",
    name="Ajuste total",
    line=dict(color="orange")
)

# Señal de Z
senial_z = go.Scatter(
    x=m_plot,
    y=N_sig * breit_wigner(m_plot, M_Z, Gamma_Z),
    mode="lines",
    name="Señal Z",
    line=dict(color="green", dash="dash")
)


fig = go.Figure(data=[hist, modelo, senial_z])

fig.update_layout(
    title="Distribución de la Masa Invariante y Ajuste del Bosón Z",
    xaxis_title="Masa Invariante [GeV/$c^2$]",
    yaxis_title="Eventos por bin (Δm = 0.5 GeV)",
    bargap=0.05,
)

fig.show()



M_Z = 90.888 ± 0.032 GeV
Gamma_Z = 4.066 ± 0.103 GeV
